# Question 2α: De-Duplication with Locality Sensitive Hashing

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import string
import numpy as np
from collections import defaultdict

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score

import re
import time
from time import perf_counter

from datasketch import MinHash, MinHashLSH

In [2]:
train_dataset = pd.read_csv('corpusTrain.csv')
test_dataset = pd.read_csv('corpusTest.csv')
train_dataset

,Id,Content
0,0,How many people are going towards using phones...
1,1,What audio format should I use for getting aud...
2,2,What is the corporate culture like at Edwards ...
3,3,What is the best barbecue in Kansas City?\n
4,4,"""Can I combine the output of two bolts to one ..."
...,...,...
531985,531985,Why is SEO important?\n
531986,531986,Who is the best employer for Robotic Process a...
531987,531987,Is it possible to cure the holes caused by pim...
531988,531988,How many employees does Infosys have?\n


In [3]:
train_dataset.dropna(inplace = True)
test_dataset.dropna(inplace = True)

# Cosine Similarity

In [4]:
# Create the Document Term Matrix
count_vec = CountVectorizer()
count_vec = count_vec.fit(train_dataset['Content'])

sparse_matrix_train = count_vec.transform(train_dataset['Content'])
sparse_matrix_test = count_vec.transform(test_dataset['Content'])

### Exact Cosine Similarity

In [ ]:
start_build_cos = time.time()

similar_items = []
counter = 0
col = ['Train Id', 'Test Id', 'Similarity']
similar_items_set= DataFrame(columns = col)

X_tfidf = []
tfidf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 3),
    min_df=0,
    stop_words='english')
X_tfidf = tfidf.fit_transform(united['Content'])


def get_similarity_items(X_tfidf, item_id, topn=3):
    """
    Get the top similar items for a given item id.
    The similarity measure here is based on cosine distance.
    """
    query = X_tfidf[item_id]
    scores = X_tfidf.dot(query.T).toarray().ravel()
    best = np.argpartition(scores, -topn)[-topn:]
    return sorted(zip(best, scores[best]), key=lambda x: -x[1])

end_build_cos = time.time()
build_time_cos = end_build_cos - start_build_cos
start_query_cos = time.time()

for i in range(X_tfidf.shape[0]-test.shape[0]):
    similar_item = get_similarity_items(X_tfidf, item_id=i)
    if similar_item[1][1] >= 0.8:
        if similar_item[1][0]>= train.shape[0]: 
            similar_items_set.at[counter, 'Train Id'] = i
            similar_items_set.at[counter, 'Test Id'] = similar_item[1][0]
            similar_items_set.at[counter, 'Similarity'] = similar_item[1][1]
            
            if similar_item[2][1] >=0.8:
                similar_items_set.at[counter, 'Train Id'] = i
                similar_items_set.at[counter, 'Test Id'] = similar_item[2][0]
                similar_items_set.at[counter, 'Similarity'] = similar_item[2][1]
            
            counter = counter + 1

end_query_cos = time.time()
query_time_cos = end_query_cos - start_query_cos
total_time_cos = build_time_cos + query_time_cos
print(similar_items_set)

### Exact Jaccard Similarity

In [ ]:
start_build_jac = time.time()

def get_shingles(text, char_ngram = 5):
    return set(text[head:head +char_ngram] for head in range(0, len(text) - char_ngram))

def jaccard(set_a, set_b):
    intersection = set_a & set_b
    union = set_a | set_b
    return len(intersection) / len(union)

shingles = []

for i_line in united['Content'].index:
    shingles.append(get_shingles(tuple(united['Content'][i_line])))
duplicates = []

end_build_jac = time.time()
build_time_jac = end_build_jac - start_build_jac

In [ ]:
#Implementation of Jaccard Similarity on the 1st 1000 samples of the corpus


start_query_jac = time.time()

for i_doc in range(len(shingles)):
    for j_doc in range(i_doc +1, len(shingles)):
        jaccard_similarity = jaccard(shingles[i_doc], shingles[j_doc])
        is_duplicate = jaccard_similarity >= 0.8
        if is_duplicate:
            duplicates.append((i_doc, j_doc, jaccard_similarity))

with pd.option_context('display.precision', 2):
    display(pd.DataFrame(duplicates, columns=['Document ID', 'Document ID', 'Jaccard Similarity']).head(n=10))

end_query_jac = time.time()        
query_time_jac = end_query_jac - start_query_jac

total_time_jac = build_time_jac + query_time_jac

# LSH Cosine Similarity

In [ ]:
start_build_lsh_cos = time.time()
def generate_random_vectors(dim, n_vectors):
    """
    generate random projection vectors
    the dims comes first in the matrix's shape,
    so we can use it for matrix multiplication.
    """
    return np.random.randn(dim, n_vectors)
def train_lsh(X_tfidf, n_vectors, seed=None):    
    if seed is not None:
        np.random.seed(seed)

    dim = X_tfidf.shape[1]
    random_vectors = generate_random_vectors(dim, n_vectors)  

    # partition data points into bins,
    # and encode bin index bits into integers
    bin_indices_bits = X_tfidf.dot(random_vectors) >= 0
    powers_of_two = 1 << np.arange(n_vectors - 1, -1, step=-1)
    bin_indices = bin_indices_bits.dot(powers_of_two)

    # update `table` so that `table[i]` is the list of document ids with bin index equal to i
    table = defaultdict(list)
    for idx, bin_index in enumerate(bin_indices):
        table[bin_index].append(idx)
    
    # note that we're storing the bin_indices here
    # so we can do some ad-hoc checking with it,
    # this isn't actually required
    model = {'table': table,
             'random_vectors': random_vectors,
             'bin_indices': bin_indices,
             'bin_indices_bits': bin_indices_bits}
    return model


# train the model
n_vectors = 16
model = train_lsh(X_tfidf, n_vectors, seed=143)

def search_nearby_bins(query_bin_bits, table, search_radius=3, candidate_set=None):
    """
    For a given query vector and trained LSH model's table
    return all candidate neighbors with the specified search radius.
    
    Example
    -------
    model = train_lsh(X_tfidf, n_vectors=16, seed=143)
    query = model['bin_index_bits'][0]  # vector for the first document
    candidates = search_nearby_bins(query, model['table'])
    """
    if candidate_set is None:
        candidate_set = set()

    n_vectors = query_bin_bits.shape[0]
    powers_of_two = 1 << np.arange(n_vectors - 1, -1, step=-1)

    for different_bits in combinations(range(n_vectors), search_radius):
        # flip the bits (n_1, n_2, ..., n_r) of the query bin to produce a new bit vector
        index = list(different_bits)
        alternate_bits = query_bin_bits.copy()
        alternate_bits[index] = np.logical_not(alternate_bits[index])

        # convert the new bit vector to an integer index
        nearby_bin = alternate_bits.dot(powers_of_two)

        # fetch the list of documents belonging to
        # the bin indexed by the new bit vector,
        # then add those documents to candidate_set;
        # make sure that the bin exists in the table
        if nearby_bin in table:
            candidate_set.update(table[nearby_bin])

    return candidate_set

def get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=3):
    table = model['table']
    random_vectors = model['random_vectors']

    # compute bin index for the query vector, in bit representation.
    bin_index_bits = np.ravel(query_vector.dot(random_vectors) >= 0)

    # search nearby bins and collect candidates
    candidate_set = set()
    for search_radius in range(max_search_radius + 1):
        candidate_set = search_nearby_bins(bin_index_bits, table, search_radius, candidate_set)

    # sort candidates by their true distances from the query
    candidate_list = list(candidate_set)
    candidates = X_tfidf[candidate_list]
    distance = pairwise_distances(candidates, query_vector, metric='cosine').flatten()
    
    distance_col = 'distance'
    nearest_neighbors = pd.DataFrame({
        'Id': candidate_list, distance_col: distance
    }).sort_values(distance_col).reset_index(drop=True)
    return nearest_neighbors
end_build_lsh_cos = time.time()
build_time_lsh_cos = end_build_lsh_cos - start_build_lsh_cos

In [ ]:
start_query_lsh_cos = time.time()

similar_items = []
counter = 0
col = ['Train Id', 'Test Id']
LSH_cos= DataFrame(columns = col)
LSH_cos_dup = DataFrame(columns = ['#Duplicates','Parameters'])

for rad in range(10):
    for item_id in range(X_tfidf.shape[0]-test_head.shape[0]):
        query_vector = X_tfidf[item_id]
        nearest_neighbors = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=rad)
        if nearest_neighbors['distance'][1]<0.165:
            if nearest_neighbors['Id'][1]>train_head.shape[0]:
                LSH_cos.at[counter, 'Train Id'] = item_id
                LSH_cos.at[counter, 'Test Id'] = nearest_neighbors.iloc[1][0]
                counter = counter + 1
    LSH_cos_dup.at[rad, '#Duplicates'] = len(LSH_cos)
    LSH_cos_dup.at[rad, '#Duplicates'] = rad
print(LSH_cos_dup)

end_query_lsh_cos = time.time()
query_time_lsh_cos = end_query_lsh_cos - start_query_lsh_cos

total_time_lsh_cos = build_time_lsh_cos + query_time_lsh_cos


# LSH Jaccard - Set number of permutations to {16,32,64}

In [4]:
train = train_dataset['Content'].str.split()
set_train = []
for i in range(len(train)):
    set_train.append(set(train[i])) 

In [5]:
test = test_dataset['Content'].str.split()
set_test = []

for i in range(len(test)):
    set_test.append(set(test[i]))

### MinHash Signatures for 16 permutations

In [39]:
#LSH Index Creation
start_buildTime_16 = time.time()

train_hashed_16 = list(range(0, train_dataset['Content'].shape[0]))
lsh_16 = MinHashLSH(threshold = 0.8, num_perm = 16)

for i in range(len(train_hashed_16)):
    train_hashed_16[i] = MinHash(num_perm = 16)
    for j in set_train[i]:
        train_hashed_16[i].update(j.encode('utf8'))
    lsh_16.insert(str(i), train_hashed_16[i])

build_time_16 = time.time() - start_buildTime_16

count_16 = 0
start_queryTime_16 = time.time()

test_hashed_16 = list(range(0, test_dataset['Content'].shape[0]))
for i in range(len(test_hashed_16)):
    test_hashed_16[i] = MinHash(num_perm = 16)
    for j in set_test[i]:
        test_hashed_16[i].update(j.encode('utf8'))

for i in range(len(test_hashed_16)):
    if len(lsh_16.query(test_hashed_16[i])):
        count_16 += 1

query_time_16 = time.time() - start_queryTime_16


In [40]:
print("Build Time: ", build_time_16)
print("Query Time: ", query_time_16)
print("Total Time: ", build_time_16 + query_time_16)
print("Duplicates: ", count_16)
print("Parameters - Number of Permutations: 16")

Build Time:  223.6151099205017
Query Time:  2.1258325576782227
Total Time:  225.74094247817993
Duplicates:  617
Parameters - Number of Permutations: 16


### MinHash Signatures for 32 permutations

In [41]:
#LSH Index Creation
start_buildTime_32 = time.time()

train_hashed_32 = list(range(0, train_dataset['Content'].shape[0]))
lsh_32 = MinHashLSH(threshold = 0.8, num_perm = 32)

for i in range(len(train_hashed_32)):
    train_hashed_32[i] = MinHash(num_perm = 32)
    for j in set_train[i]:
        train_hashed_32[i].update(j.encode('utf8'))
    lsh_32.insert(str(i), train_hashed_32[i])

build_time_32 = time.time() - start_buildTime_32

count_32 = 0
start_queryTime_32 = time.time()

test_hashed_32 = list(range(0, test_dataset['Content'].shape[0]))
for i in range(len(test_hashed_32)):
    test_hashed_32[i] = MinHash(num_perm = 32)
    for j in set_test[i]:
        test_hashed_32[i].update(j.encode('utf8'))

for i in range(len(test_hashed_32)):
    if len(lsh_32.query(test_hashed_32[i])):
        count_32 += 1

query_time_32 = time.time() - start_queryTime_32


In [42]:
print("Build Time: ", build_time_32)
print("Query Time: ", query_time_32)
print("Total Time: ", build_time_32 + query_time_32)
print("Duplicates: ", count_32)
print("Parameters - Number of Permutations: 32")

Build Time:  277.75269055366516
Query Time:  2.739528179168701
Total Time:  280.49221873283386
Duplicates:  500
Parameters - Number of Permutations: 32


### MinHash Signatures for 64 permutations

In [43]:
#LSH Index Creation
start_buildTime_64 = time.time()

train_hashed_64 = list(range(0, train_dataset['Content'].shape[0]))
lsh_64 = MinHashLSH(threshold = 0.8, num_perm = 64)

for i in range(len(train_hashed_64)):
    train_hashed_64[i] = MinHash(num_perm = 64)
    for j in set_train[i]:
        train_hashed_64[i].update(j.encode('utf8'))
    lsh_64.insert(str(i), train_hashed_64[i])

build_time_64 = time.time() - start_buildTime_64

count_64 = 0
start_queryTime_64 = time.time()

test_hashed_64 = list(range(0, test_dataset['Content'].shape[0]))
for i in range(len(test_hashed_64)):
    test_hashed_64[i] = MinHash(num_perm = 64)
    for j in set_test[i]:
        test_hashed_64[i].update(j.encode('utf8'))

for i in range(len(test_hashed_64)):
    if len(lsh_64.query(test_hashed_64[i])):
        count_64 += 1

query_time_64 = time.time() - start_queryTime_64


In [44]:
print("Build Time: ", build_time_64)
print("Query Time: ", query_time_64)
print("Total Time: ", build_time_64 + query_time_64)
print("Duplicates: ", count_64)
print("Parameters - Number of Permutations: 64")

Build Time:  422.53412795066833
Query Time:  3.790764331817627
Total Time:  426.32489228248596
Duplicates:  541
Parameters - Number of Permutations: 64
